# 10x scRNA-seq gene expression data

To efficiently handle data transfer and download this large > 4 million cell dataset, we provide the 10x gene expression data divided into 19 packages where cells are grouped by method (10Xv2, 10Xv3) and the broad anatomical region the cells were dissected from.

The purpose of this notebook is to provide an overview of the file organization and how to recombine the data through example use cases.

In [1]:
import os
import pandas as pd
import numpy as np
import anndata
import time

In [2]:
input_base = '/allen/programs/celltypes/workgroups/rnaseqanalysis/lydian/ABC_handoff'
input_directory = os.path.join( input_base, 'metadata', '20230630', '10x-scRNA-seq' )

view_directory = os.path.join( input_directory, 'views')
cache_views = False
if cache_views :
    os.makedirs( view_directory, exist_ok=True )

## Data overview
### Cell metadata

Essential cell metadata is stored as a dataframe. Each row represents one cell indexed by a *cell label*. The *cell label* is the concatenation of barcode and name of the sample. In this context, the sample is the *barcoded cell sample* that represents a single load into one port of the 10x Chromium. Note that the barcode is only unique within a single *barcoded cell sample* and that the same barcode can be reused. The *barcoded cell sample* label or name is unique in the database.

Each cell is associated with a *library label*, *library method*, *donor label*, *donor genotype*, *donor sex*, dissection *region of interest acronym", the corresponding coarse *anatomical division label* and the *matrix_prefix* identifying which data package this cell is part of. Further, each cell is associated with a *cluster alias* representing which cluster this cell is a member of.


In [3]:
file = os.path.join( input_directory, 'cell_metadata.csv')
cell = pd.read_csv(file)
cell.set_index('cell_label',inplace=True)
print("Number of cells = ", len(cell))
cell.head(5)

Number of cells =  4057701


,library_label,anatomical_division_label,cluster_alias,library_method,region_of_interest_acronym,donor_label,donor_genotype,donor_sex,matrix_prefix
cell_label,,,,,,,,,
GACTGCGAGCTGCAAG-020_C01,L8TX_180406_01_F01,HPF,1,10Xv2,RHP,Snap25-IRES2-Cre;Ai14-381293,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,M,WMB-10Xv2-HPF
CGATGTAGTGCTAGCC-021_D01,L8TX_180406_01_C02,HPF,1,10Xv2,RHP,Snap25-IRES2-Cre;Ai14-381295,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,F,WMB-10Xv2-HPF
CACATAGCATGGTTGT-020_D01,L8TX_180406_01_B02,HPF,1,10Xv2,RHP,Snap25-IRES2-Cre;Ai14-381293,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,M,WMB-10Xv2-HPF
CCTAAAGGTGACCAAG-020_C01,L8TX_180406_01_F01,HPF,1,10Xv2,RHP,Snap25-IRES2-Cre;Ai14-381293,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,M,WMB-10Xv2-HPF
CGGAGCTAGCTTTGGT-021_D01,L8TX_180406_01_C02,HPF,1,10Xv2,RHP,Snap25-IRES2-Cre;Ai14-381295,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,F,WMB-10Xv2-HPF


We can use panadas groupby function to see how many unique items are associated for each field and list them out if the number of items is small.

In [4]:
def print_column_info(df) :
    
    for c in df.columns :
        grouped = df[[c]].groupby(c).count()
        members = ''
        if len(grouped) < 30 :
            members = str(list(grouped.index))
        print("Number of unique %s = %d %s" % (c,len(grouped),members))
        

In [5]:
print_column_info(cell)

Number of unique library_label = 781 
Number of unique anatomical_division_label = 12 ['CB', 'CTXsp', 'HPF', 'HY', 'Isocortex', 'MB', 'MY', 'OLF', 'P', 'PAL', 'STR', 'TH']
Number of unique cluster_alias = 5196 
Number of unique library_method = 2 ['10Xv2', '10Xv3']
Number of unique region_of_interest_acronym = 29 ['ACA', 'AI', 'AUD', 'AUD-TEa-PERI-ECT', 'CB', 'CTXsp', 'ENT', 'HIP', 'HY', 'LSX', 'MB', 'MO-FRP', 'MOp', 'MY', 'OLF', 'P', 'PAL', 'PL-ILA-ORB', 'RHP', 'RSP', 'SS-GU-VISC', 'SSp', 'STRd', 'STRv', 'TEa-PERI-ECT', 'TH', 'VIS', 'VIS-PTLp', 'sAMY']
Number of unique donor_label = 318 
Number of unique donor_genotype = 4 ['Ai14(RCL-tdT)/wt', 'Gad2-IRES-Cre/wt;Ai14(RCL-tdT)/wt', 'Slc32a1-IRES-Cre/wt;Ai14(RCL-tdT)/wt', 'Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt']
Number of unique donor_sex = 2 ['F', 'M']
Number of unique matrix_prefix = 19 ['WMB-10Xv2-CTXsp', 'WMB-10Xv2-HPF', 'WMB-10Xv2-HY', 'WMB-10Xv2-Isocortex', 'WMB-10Xv2-MB', 'WMB-10Xv2-OLF', 'WMB-10Xv2-TH', 'WMB-10Xv3-CB', 'WMB-10Xv3-

We can also bring in the pivot table from the "cluster annotation tutorial" to associate each cell with terms at each cell type classification level.

In [6]:
file = os.path.join( input_directory, '..', 'cluster_annotation', 'views',
             'cluster_to_cluster_annotation_membership_pivoted.csv')
cluster_details = pd.read_csv(file)
cluster_details.set_index('cluster_alias', inplace=True)
cluster_details.head(5)

,neurotransmitter,division,class,subclass,supertype,cluster
cluster_alias,,,,,,
1,Glut,1 Pallium glutamatergic,01 IT-ET Glut,018 L2 IT PPP-APr Glut,0081 L2 IT PPP-APr Glut_3,0327 L2 IT PPP-APr Glut_3
2,Glut,1 Pallium glutamatergic,01 IT-ET Glut,018 L2 IT PPP-APr Glut,0081 L2 IT PPP-APr Glut_3,0328 L2 IT PPP-APr Glut_3
3,Glut,1 Pallium glutamatergic,01 IT-ET Glut,018 L2 IT PPP-APr Glut,0080 L2 IT PPP-APr Glut_2,0323 L2 IT PPP-APr Glut_2
4,Glut,1 Pallium glutamatergic,01 IT-ET Glut,018 L2 IT PPP-APr Glut,0080 L2 IT PPP-APr Glut_2,0324 L2 IT PPP-APr Glut_2
5,Glut,1 Pallium glutamatergic,01 IT-ET Glut,018 L2 IT PPP-APr Glut,0080 L2 IT PPP-APr Glut_2,0326 L2 IT PPP-APr Glut_2


In [7]:
cell_extended = cell.join(cluster_details,on='cluster_alias')
cell_extended.head(5)

,library_label,anatomical_division_label,cluster_alias,library_method,region_of_interest_acronym,donor_label,donor_genotype,donor_sex,matrix_prefix,neurotransmitter,division,class,subclass,supertype,cluster
cell_label,,,,,,,,,,,,,,,
GACTGCGAGCTGCAAG-020_C01,L8TX_180406_01_F01,HPF,1,10Xv2,RHP,Snap25-IRES2-Cre;Ai14-381293,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,M,WMB-10Xv2-HPF,Glut,1 Pallium glutamatergic,01 IT-ET Glut,018 L2 IT PPP-APr Glut,0081 L2 IT PPP-APr Glut_3,0327 L2 IT PPP-APr Glut_3
CGATGTAGTGCTAGCC-021_D01,L8TX_180406_01_C02,HPF,1,10Xv2,RHP,Snap25-IRES2-Cre;Ai14-381295,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,F,WMB-10Xv2-HPF,Glut,1 Pallium glutamatergic,01 IT-ET Glut,018 L2 IT PPP-APr Glut,0081 L2 IT PPP-APr Glut_3,0327 L2 IT PPP-APr Glut_3
CACATAGCATGGTTGT-020_D01,L8TX_180406_01_B02,HPF,1,10Xv2,RHP,Snap25-IRES2-Cre;Ai14-381293,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,M,WMB-10Xv2-HPF,Glut,1 Pallium glutamatergic,01 IT-ET Glut,018 L2 IT PPP-APr Glut,0081 L2 IT PPP-APr Glut_3,0327 L2 IT PPP-APr Glut_3
CCTAAAGGTGACCAAG-020_C01,L8TX_180406_01_F01,HPF,1,10Xv2,RHP,Snap25-IRES2-Cre;Ai14-381293,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,M,WMB-10Xv2-HPF,Glut,1 Pallium glutamatergic,01 IT-ET Glut,018 L2 IT PPP-APr Glut,0081 L2 IT PPP-APr Glut_3,0327 L2 IT PPP-APr Glut_3
CGGAGCTAGCTTTGGT-021_D01,L8TX_180406_01_C02,HPF,1,10Xv2,RHP,Snap25-IRES2-Cre;Ai14-381295,Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt,F,WMB-10Xv2-HPF,Glut,1 Pallium glutamatergic,01 IT-ET Glut,018 L2 IT PPP-APr Glut,0081 L2 IT PPP-APr Glut_3,0327 L2 IT PPP-APr Glut_3


In [8]:
print_column_info(cell_extended)

Number of unique library_label = 781 
Number of unique anatomical_division_label = 12 ['CB', 'CTXsp', 'HPF', 'HY', 'Isocortex', 'MB', 'MY', 'OLF', 'P', 'PAL', 'STR', 'TH']
Number of unique cluster_alias = 5196 
Number of unique library_method = 2 ['10Xv2', '10Xv3']
Number of unique region_of_interest_acronym = 29 ['ACA', 'AI', 'AUD', 'AUD-TEa-PERI-ECT', 'CB', 'CTXsp', 'ENT', 'HIP', 'HY', 'LSX', 'MB', 'MO-FRP', 'MOp', 'MY', 'OLF', 'P', 'PAL', 'PL-ILA-ORB', 'RHP', 'RSP', 'SS-GU-VISC', 'SSp', 'STRd', 'STRv', 'TEa-PERI-ECT', 'TH', 'VIS', 'VIS-PTLp', 'sAMY']
Number of unique donor_label = 318 
Number of unique donor_genotype = 4 ['Ai14(RCL-tdT)/wt', 'Gad2-IRES-Cre/wt;Ai14(RCL-tdT)/wt', 'Slc32a1-IRES-Cre/wt;Ai14(RCL-tdT)/wt', 'Snap25-IRES2-Cre/wt;Ai14(RCL-tdT)/wt']
Number of unique donor_sex = 2 ['F', 'M']
Number of unique matrix_prefix = 19 ['WMB-10Xv2-CTXsp', 'WMB-10Xv2-HPF', 'WMB-10Xv2-HY', 'WMB-10Xv2-Isocortex', 'WMB-10Xv2-MB', 'WMB-10Xv2-OLF', 'WMB-10Xv2-TH', 'WMB-10Xv3-CB', 'WMB-10Xv3-

For convenience, we can cache this view for later reuse. 

In [9]:
if cache_views :
    file = os.path.join( view_directory, 'cell_metadata_with_cluster_annotation.csv')
    cell_extended.to_csv( file )

### Gene expression matrices

The > 4 million datasets has been subdivided into 19 packages to data transfer and download more efficient. Each package is formatted as annadata h5ad file with minimal metadata. In this next section, we provide example code on how to open the file and  connect with the rich cell level metadata discussed above.

For each subset, there are two h5ad files one storing the raw counts and the other log normalization of it. The file name has the pattern "matrix_prefix-normalization.h5ad".

In [10]:
expression_directory = os.path.join(input_base, '10x')
matrix_prefix = 'WMB-10Xv2-TH'
ext = 'h5ad'
normalization = 'log2'

file = os.path.join( expression_directory, '%s-%s.%s'% (matrix_prefix,normalization,ext) )
print(file)

/allen/programs/celltypes/workgroups/rnaseqanalysis/lydian/ABC_handoff/10x/WMB-10Xv2-TH-log2.h5ad


We use the anndata's *read_h5ad* function to open the package for the log2 normalization file. The "backed='r'" makes use of the lazy loading functionality to only load the required data. By default, anndata will load the entire expression matrix in memory.

In [11]:
adata = anndata.read_h5ad(file,backed='r')
print(adata)

AnnData object with n_obs × n_vars = 131212 × 32285 backed at '/allen/programs/celltypes/workgroups/rnaseqanalysis/lydian/ABC_handoff/10x/WMB-10Xv2-TH-log2.h5ad'
    obs: 'cell_barcode', 'library_label', 'anatomical_division_label'
    var: 'gene_symbol'
    uns: 'normalization', 'parent', 'parent_layer', 'parent_rows'


Genes are represented as "variables". For this data, the *var* dataframe is indexed by the Ensembl gene identifier with one metadata column *gene symbol*.

In [12]:
print("Number of genes = ", len(adata.var))
adata.var.index[0:5]

Number of genes =  32285


Index(['ENSMUSG00000051951', 'ENSMUSG00000089699', 'ENSMUSG00000102331',
       'ENSMUSG00000102343', 'ENSMUSG00000025900'],
      dtype='object', name='gene_identifier')

Cells are represented as "observations". For this data, the *obs* dataframe is index by *cell label* with three minimal metadata columns *cell barcode*, *library label* and *anatomical division label*. 

In [13]:
print("Number of cells = ", len(adata.obs))
adata.obs.index[0:5]

Number of cells =  131212


Index(['CAGGTGCAGGCTAGCA-040_C01', 'TGCGCAGGTTGCGCAC-045_C01',
       'CGATGTATCTTGCCGT-042_B01', 'GACTAACGTCCTCTTG-040_B01',
       'GATCGTACAACTGCTA-040_B01'],
      dtype='object', name='cell_label')

We can easily connect the cells in the package to the extended cell metadata with simple pandas filtering.

In [14]:
cell_filtered = cell_extended.loc[adata.obs.index]
print("Number of cells = ", len(cell_filtered))
print_column_info( cell_filtered )

Number of cells =  131212
Number of unique library_label = 26 ['L8TX_180815_01_A07', 'L8TX_180815_01_A08', 'L8TX_180815_01_B07', 'L8TX_180815_01_B08', 'L8TX_180815_01_C07', 'L8TX_180815_01_C08', 'L8TX_180815_01_D07', 'L8TX_180815_01_D08', 'L8TX_180815_01_E07', 'L8TX_180815_01_E08', 'L8TX_180815_01_F07', 'L8TX_180815_01_F08', 'L8TX_180815_01_G07', 'L8TX_180815_01_G08', 'L8TX_180815_01_H07', 'L8TX_180815_01_H08', 'L8TX_180829_01_A09', 'L8TX_180829_01_B09', 'L8TX_180829_01_C09', 'L8TX_180829_01_C10', 'L8TX_180829_01_E09', 'L8TX_180829_01_F09', 'L8TX_180829_01_G09', 'L8TX_190321_01_F03', 'L8TX_190321_01_G03', 'L8TX_190321_01_H03']
Number of unique anatomical_division_label = 1 ['TH']
Number of unique cluster_alias = 1082 
Number of unique library_method = 1 ['10Xv2']
Number of unique region_of_interest_acronym = 1 ['TH']
Number of unique donor_label = 8 ['Snap25-IRES2-Cre;Ai14-404122', 'Snap25-IRES2-Cre;Ai14-404123', 'Snap25-IRES2-Cre;Ai14-404124', 'Snap25-IRES2-Cre;Ai14-404125', 'Snap25-I

## Example use cases

In this section, we explore two use cases. The first example looks at the expression of nine canonical neurotransmitter transporter genes and the second the expression of gene Tac2.

To support these use cases, we will create a smaller submatrix (all cells and 10 genes) that read it into memory. *Note this operation takes around 10 seconds*.

In [15]:
ntgenes = ['Slc17a7','Slc17a6','Slc17a8','Slc32a1','Slc6a5','Slc18a3','Slc6a3','Slc6a4','Slc6a2']
exgenes = ['Tac2']
gnames = ntgenes + exgenes
pred = [x in gnames for x in adata.var.gene_symbol]
gene_filtered = adata.var[pred]
gene_filtered

,gene_symbol
gene_identifier,
ENSMUSG00000037771,Slc32a1
ENSMUSG00000070570,Slc17a7
ENSMUSG00000039728,Slc6a5
ENSMUSG00000030500,Slc17a6
ENSMUSG00000055368,Slc6a2
ENSMUSG00000019935,Slc17a8
ENSMUSG00000025400,Tac2
ENSMUSG00000020838,Slc6a4
ENSMUSG00000021609,Slc6a3


In [16]:
start = time.process_time()
asubset = adata[:,gene_filtered.index].to_memory()
print("time taken: ", time.process_time() - start)
print(asubset)

time taken:  3.5271203420000017
AnnData object with n_obs × n_vars = 131212 × 10
    obs: 'cell_barcode', 'library_label', 'anatomical_division_label'
    var: 'gene_symbol'
    uns: 'normalization', 'parent', 'parent_layer', 'parent_rows'


We define two helper functions to (1) *create_expression_dataframe*: create joined gene expression and cell metadata dataframe based for a set of input genes and (2) *aggregate_by_metadata* : compute the average expression for a given catergory.

In [17]:
def create_expression_dataframe( ad, gf ) :
    gdata = ad[:,gf.index].to_df()
    gdata.columns = gf.gene_symbol
    joined = cell_filtered.join( gdata )
    return joined

In [18]:
def aggregate_by_metadata( df, gnames, value ) :
    grouped = df.groupby(value)[gnames].mean()
    grouped = grouped.sort_values(by=gnames[0],ascending=False)
    return grouped

### Expression of canonical neurotransmitter transporter genes in the thalamus

During analysis, clusters were assigned neurotransmitter identities based on the expression of of canonical neurotransmitter transporter genes: Slc17a7 (also known as Vglut1), Slc17a6 (Vglut2) and Slc17a8
(Vglut3) for glutamatergic, Slc32a1 (Vgat) for GABAergic, Slc6a5 for glycinergic, Slc18a3 (Vacht) for cholinergic, Slc6a3 (Dat) for dopaminergic, Slc6a4 (Sert) for serotonergic, and 
Slc6a2 (Net) for noradrenergic.

In this example, we create a dataframe comprising of cells in the 'WMB-10Xv2-TH' package and expression of the 9 solute carrier family genes above. We then group the cells by the assigned neurotransmitter class and compute the mean expression for each group and visualized as a colorized table. As expected, we see enriched gene expression for the associated neurotransmitter class describe above with varying degree of specificity.

In [19]:
pred = [x in ntgenes for x in asubset.var.gene_symbol]
gf = asubset.var[pred]

ntexp = create_expression_dataframe( asubset, gf )
agg = aggregate_by_metadata(ntexp, gf.gene_symbol, 'neurotransmitter').head(10)
agg.style.background_gradient(cmap='Reds')

,Slc32a1,Slc17a7,Slc6a5,Slc17a6,Slc6a2,Slc17a8,Slc6a4,Slc6a3,Slc18a3
neurotransmitter,,,,,,,,,
GABA,7.164396,0.419043,0.026049,1.216959,0.001380,0.032728,0.011204,0.016119,0.000960
GABA-Glyc,6.367074,0.292678,3.755787,1.885634,0.000000,0.000000,0.000000,0.125798,0.000000
Glut-GABA,5.473776,0.000000,0.000000,4.691283,0.000000,0.000000,0.000000,0.000000,0.000000
Dopa,0.757809,0.604825,0.000000,3.615892,0.000000,0.000000,0.000000,5.524069,0.000000
Hist,0.108541,0.155022,0.000000,1.174037,0.000000,0.052135,0.000000,0.000000,0.000000
Chol,0.087876,8.608787,0.005719,6.528627,0.000000,0.042500,0.011684,0.000000,3.322996
Glut,0.083664,3.403658,0.035956,8.989318,0.001436,0.233407,0.009094,0.003759,0.025051


### Expression of Tachykinin 2 (Tac2) in the thalamus

In mice, the tachykinin 2 (Tac2) gene encodes neuropeptide called neurokinin B (NkB). Tac2 is produced by neurons in specific regions of the brain know to be invovled in emotion and social behavior. Based on [ISH data](https://mouse.brain-map.org/experiment/siv?id=77279001&imageId=77284584&initImage=ish&coordSystem=pixel&x=5384.5&y=3832.5&z=3) from the Allen Mouse Brain Atlas, Tac 2 is sparsely expressed in the mouse isocortex and densely enriched is specific subcortical regions such the medial habenula (MH), the amygdala and hypothalamus.

![Expression of Tac2 in an adult mouse](Tac2_ISH_Coronal_Section.JPG)

In this example, we create a dataframe comprising of cells in the 'WMB-10Xv2-TH' package and expression values of Tac2 in thouse cells.


In [20]:
# filtered to gene of interest
gf = asubset.var[asubset.var.gene_symbol == 'Tac2']
tac2 = create_expression_dataframe( asubset, gf )

Grouping cells by neurotransmitter identites and computing the mean expression in each group, we can observed that Tac2 gene in highly enriched in cholinergic cell types.

In [21]:
agg = aggregate_by_metadata(tac2, gf.gene_symbol, 'neurotransmitter').head(10)
agg.style.background_gradient(cmap='Reds')

,Tac2
neurotransmitter,
Chol,7.225159
Glut,0.191328
GABA-Glyc,0.129728
Hist,0.055357
GABA,0.041769
Dopa,0.000000
Glut-GABA,0.000000


Grouping cells by cell types class, shows that Tac2 is enriched in class "08 MH-LH Glut" with cells restricted to the medial (MH) and lateral (LH) habenula and mixture of glutamatergic and cholinergic types.

In [22]:
agg = aggregate_by_metadata(tac2, gf.gene_symbol, 'class').head(8)
agg.style.background_gradient(cmap='Reds')

,Tac2
class,
08 MH-LH Glut,4.477652
22 MY GABA,0.540533
01 IT-ET Glut,0.424430
17 P Glut,0.384413
06 CNU GABA,0.114376
27 Pineal Glut,0.098581
28 Astro-Epen,0.071376
16 MB Glut,0.066691


At the next level, grouping by subclass, shows Tac2 is enriched in sublcass "063 MH Tac2 Glut" which specificity to the medial habenula and with Tac2 itself as a marker for the subclass and multiple children supertypes.

In [23]:
agg = aggregate_by_metadata(tac2, gf.gene_symbol, 'subclass').head(8)
agg.style.background_gradient(cmap='Reds')

,Tac2
subclass,
063 MH Tac2 Glut,6.219506
171 PB-an-do Evx2 Glut,3.459718
193 PAS-MV Ebf2 Gly-Gaba,1.297280
197 PAG-MRN Rln3 Gaba,1.260279
045 STRv Lhx8 Gaba,0.944707
047 NDB-SI-MA-STRv Lhx8 Gaba,0.646565
293 CHOR NN,0.435719
012 MEA Slc17a7 Glut,0.424430


In [24]:
agg = aggregate_by_metadata(tac2, gf.gene_symbol, 'supertype').head(8)
agg.style.background_gradient(cmap='Reds')

,Tac2
supertype,
0249 MH Tac2 Glut_2,7.927101
0700 PB-an-do Evx2 Glut_4,6.919436
0248 MH Tac2 Glut_1,6.699828
0250 MH Tac2 Glut_3,6.265915
0182a NDB-SI-MA-STRv Lhx8 Gaba_8,3.556106
0251 MH Tac2 Glut_4,2.401563
0172 STRv Lhx8 Gaba_1,1.322590
0783 PAS-MV Ebf2 Gly-Gaba_1,1.297280


In [25]:
adata.file.close()